# Projet Spark - Covid VS Societé 2020

## Problematique
### Quelle est l'impacte du covid sur les société en France sur l'année 2020 ?


In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import *
import pyspark.sql.functions as f

**Création de la session spark**

In [2]:
spark = SparkSession.builder.appName("covid_societe").getOrCreate()

**Lecture du fichier des données covid-19**

In [3]:
path_file_data = "../data/donnees_covid_societes/"

df_covid_par_dep_et_sexe_patient = spark.read.csv(
    path_file_data + "covid/covid_par_dep_et_sexe_du_patient/donnees-hospitalieres-covid19-2020-12-13-19h03.csv",
    header=True,
    sep=";"
)
df_metadonne_covid_par_dep_et_sexe_patient = spark.read.csv(
    path_file_data + "covid/covid_par_dep_et_sexe_du_patient/metadonnees-donnees-hospitalieres-covid19.csv",
    header=True,
    sep=";"
)

# Affichage des metadonnés
df_metadonne_covid_par_dep_et_sexe_patient.select(
    "Colonne",
    "Type ",
    "Description_FR"
).show(20, False)
# Affichage des données
df_covid_par_dep_et_sexe_patient.show(20)

# La plus grande date
df_covid_par_dep_et_sexe_patient.orderBy('jour', ascending=False).show(1)

+-------+-------------+------------------------------------------------------------------+
|Colonne|Type         |Description_FR                                                    |
+-------+-------------+------------------------------------------------------------------+
|dep    |integer      |Departement                                                       |
|sexe   |integer      |Sexe                                                              |
|jour   |string($date)|Date de notification                                              |
|hosp   |integer      |Nombre de personnes actuellement hospitalisées                    |
|rea    |integer      |Nombre de personnes actuellement en réanimation ou soins intensifs|
|rad    |integer      |Nombre cumulé de personnes retournées à domicile                  |
|dc     |integer      |Nombre cumulé de personnes décédées à l'hôpital                   |
+-------+-------------+------------------------------------------------------------------+

**On supprime le sexe du df et on regroupe par dep et jour**

In [4]:
from pyspark.sql.types import IntegerType
#On regroupe les données par departements et par jour
df_covid_dep_patient = df_covid_par_dep_et_sexe_patient.groupBy(
        "dep",
        "jour"
    ).agg( 
        # On fait la somme de chaque colonne puis on cast le resultat en INTEGER et on le renomme
        f.sum("hosp").cast(IntegerType()).alias('hosp'), 
        f.sum("rea").cast(IntegerType()).alias('rea'),
        f.sum("rad").cast(IntegerType()).alias('rad'),
        f.sum("dc").cast(IntegerType()).alias('dc')
    )

df_covid_dep_patient = df_covid_dep_patient.orderBy(["jour","dep"], ascending=True)

df_covid_dep_patient.show()

+---+----------+----+---+---+---+
|dep|      jour|hosp|rea|rad| dc|
+---+----------+----+---+---+---+
| 01|2020-03-18|   4|  0|  2|  0|
| 02|2020-03-18|  82| 20| 36| 22|
| 03|2020-03-18|   8|  0|  2|  0|
| 04|2020-03-18|   6|  2|  4|  0|
| 05|2020-03-18|  16|  2| 18|  0|
| 06|2020-03-18|  50|  2| 94|  4|
| 07|2020-03-18|  22|  2|  0|  0|
| 08|2020-03-18|   0|  0|  2|  0|
| 09|2020-03-18|   2|  2|  4|  0|
| 10|2020-03-18|  10|  0|  0|  0|
| 11|2020-03-18|  16| 14| 18|  6|
| 12|2020-03-18|  14|  4|  2|  0|
| 13|2020-03-18| 196| 22|117|  8|
| 14|2020-03-18|   8|  2|  0|  0|
| 15|2020-03-18|   4|  2|  0|  0|
| 16|2020-03-18|   2|  0|  0|  0|
| 17|2020-03-18|  10|  4|  4|  0|
| 18|2020-03-18|   0|  0|  0|  0|
| 19|2020-03-18|   0|  0|  0|  0|
| 21|2020-03-18| 120| 32|  8|  0|
+---+----------+----+---+---+---+
only showing top 20 rows



### Quelle departement a eu le plus d'hospitalisation, de réanimation et de déces ? (ecrire dans un csv dans /resultat)

In [5]:
df_covid_sum_par_dep = df_covid_dep_patient.groupBy(
    "dep"
).agg( 
    # On fait la somme de chaque colonne puis on cast le resultat en INTEGER et on le renomme
    f.sum("hosp").cast(IntegerType()).alias('hosp'), 
    f.sum("rea").cast(IntegerType()).alias('rea'),
    f.sum("rad").cast(IntegerType()).alias('rad'),
    f.sum("dc").cast(IntegerType()).alias('dc')
).orderBy("dep")

df_covid_sum_par_dep.show()

df_covid_max_hospi_par_dep = df_covid_sum_par_dep.groupBy().max("hosp")
df_covid_max_hospi_par_dep.show()

+---+------+-----+-------+------+
|dep|  hosp|  rea|    rad|    dc|
+---+------+-----+-------+------+
| 01| 66400| 5459| 262394| 63392|
| 02| 72338| 8637| 547503|143384|
| 03| 31927| 3764| 136395| 31031|
| 04| 16061| 1240|  84311|  9768|
| 05| 24904| 3088|  88481| 14146|
| 06| 92538|13738| 489842|109182|
| 07| 36817| 3964| 235889| 62359|
| 08| 22346| 3103| 111299| 33068|
| 09|  6017| 1227|  22829|  2296|
| 10| 40407| 3493| 316600| 70447|
| 11| 16175| 2281| 129024| 33274|
| 12| 17870| 1584|  88938| 17852|
| 13|368942|63418|2246573|345858|
| 14| 47116| 7988| 196643| 48947|
| 15| 12643| 2032|  35388|  6900|
| 16|  6111| 1121|  36016|  7938|
| 17| 20872| 3781|  87128| 28192|
| 18| 30692| 1984| 107314| 48841|
| 19| 14738| 1986|  84227| 21255|
| 21| 67218|12652| 497990|132616|
+---+------+-----+-------+------+
only showing top 20 rows

+---------+
|max(hosp)|
+---------+
|   547599|
+---------+



### Combien d'homme, femme ou autre, on etait hospitalié par le covid (par departement) ? (ecrire dans un csv dans /resultat) (Louis)
DF1: dep | sexe | sum(hospi)

      01 | 0    | 120 000
      
      01 | 1    | 120 000
      
      01 | 2    | 160 000
      
      02 | 0    | 160 000

### Quelle jour a été le plus critique en hospitalisation, reanimation et deces (par departement) (ecrire dans un csv dans /resultat)
DF1: dep | jour | max(hospi)

DF2: dep | jour | max(rea)

DF3: dep | jour | max(deces)

### Quelle est l'impacte du covid sur les société en France sur l'année 2020 ?

DF societe : dep | date radiation | nb_de_societe_radié

DF covid : dep   | jour           | hosp | rea | rad | dc

Df covid_societe : dep | date | nb_de_societe_radié | hosp | rea | rad | dc